In [ ]:
import os
import cv2
import numpy as np
import math
from matplotlib import pyplot as plt
from PIL import ImageEnhance, Image
import statistics as s
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib_scalebar.scalebar import ScaleBar

In [ ]:
folder = input('Which folder are your images located in? ')
os.chdir('C:\\Users\\abhim\\Desktop\\'+folder)
print('All Set!')

In [ ]:
%store -r lowerColorArray
%store -r upperColorArray
%store -r meanSizeList
%store -r minSizeList
%store -r conSizeList
%store -r alpha_mult
%store -r beta_sub
%store -r mean_inten

In [ ]:
# print(lowerColorArray, "\n\n", upperColorArray)

global cells_count, color_outline
global mask, opening, close, cnts
global count_live, count_early, count_late, count_nec
global alpha_this
global live_means, early_means, late_means, nec_means, totals, early_per, late_per, nec_per
global numDataSets, numImg
global printorig
global imgList, maskList

alpha_this = 1
live_means, early_means, late_means, nec_means = [],[],[],[]
totals, early_per, late_per, nec_per = [],[],[],[]
live_stdevs, early_stdevs, late_stdevs, nec_stdevs = [],[],[],[]
numDataSets,numImg = 0,0

In [ ]:
numDataSets = int(input('How many datasets are you working with? '))
print('Cool!')
numImg = int(input('How many images in each dataset? '))
print('Awesome!')
imgList, maskList = [[[0 for i in range(6)] for j in range(numImg)] for k in range(numDataSets)], [[[0 for i in range(4)] for j in range(numImg)] for k in range(numDataSets)]
count_live = [[0 for i in range(numImg)] for j in range(numDataSets)]
count_early = [[0 for i in range(numImg)] for j in range(numDataSets)]
count_late = [[0 for i in range(numImg)] for j in range(numDataSets)]
count_nec = [[0 for i in range(numImg)] for j in range(numDataSets)]
setCounts = [count_live, count_early, count_late, count_nec]


In [ ]:
for rotDataSets in range(numDataSets):
    for rotImg in range(numImg):    
        redChannel = cv2.imread('r'+(str(rotDataSets+1))+'_'+(str(rotImg+1))+'.tif')
        greenChannel = cv2.imread('g'+(str(rotDataSets+1))+'_'+(str(rotImg+1))+'.tif')
        greenChannelJet = cv2.imread('g'+(str(rotDataSets+1))+'_'+(str(rotImg+1))+'.tif',0)
        #cv2.multiply(greenChannel, 1.2, greenChannel)
        
        imgList[rotDataSets][rotImg][0] = greenChannel # green Channel
        imgList[rotDataSets][rotImg][1] = greenChannelJet # green Channel Jet
        
        image = cv2.cvtColor(redChannel+greenChannel, cv2.COLOR_BGR2RGB)
        printorig = image.copy()
        
        imgList[rotDataSets][rotImg][2] = printorig # original image
        
        cv2.multiply(image, alpha_mult, image)
        cv2.add(image, beta_sub, image)
        image = Image.fromarray(image)
        enhancer = ImageEnhance.Sharpness(image)
        factor = 3.6
        image = enhancer.enhance(factor)
        image = np.asarray(image)
        cv2.add(image, beta_sub, image)
        image = cv2.bilateralFilter(image,9,75,75)
        
        ############ USE only if necessary! ####################
        avg_intens = image.mean(axis=0).mean(axis=0)
        mean_inten_this = 0.299*avg_intens[0] + 0.587*avg_intens[1] + 0.114*avg_intens[2]
        alpha_this = mean_inten/mean_inten_this
        cv2.multiply(image, np.array([alpha_this]), image)
        ########################################################
        
        imgList[rotDataSets][rotImg][3] = image # enhanced image
        
        original = image.copy()
        
        cells_count = [0,0,0,0]        
        color_outline = [[0,255,0],[255,255,0],[205,125,0],[255,0,0]]
        mask = [0,0,0,0]
        opening = [0,0,0,0]
        close = [0,0,0,0]
        cnts = [0,0,0,0]

        for listVar in range(len(cells_count)):
            mask[listVar] = cv2.inRange(original, lowerColorArray[listVar], upperColorArray[listVar]) #  cv2.Canny(image_max,95,95) #
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
            opening[listVar] = cv2.morphologyEx(mask[listVar], cv2.MORPH_OPEN, kernel, iterations=0) 
            close[listVar] = cv2.morphologyEx(opening[listVar], cv2.MORPH_CLOSE, kernel, iterations=2)
            cnts = cv2.findContours(close[listVar], cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            for c in cnts:
                area = cv2.contourArea(c)
                if area > minSizeList[listVar] and meanSizeList[listVar] > 0:
                    cv2.drawContours(original, [c], -1, color_outline[listVar], 6) #36,255,12
        
        imgList[rotDataSets][rotImg][4] = original # contour image
        
        lowerColorArray2 = np.array([[0,250,0],[250,250,0],[200,120,0],[250,0,0]])
        upperColorArray2 = np.array([[1,255,1],[255,255,1],[205,125,1],[255,1,0]])
        
        for listVar in range(len(cells_count)):
            mask[listVar] = cv2.inRange(original, lowerColorArray2[listVar], upperColorArray2[listVar]) #  cv2.Canny(image_max,95,95) #
            
            maskList[rotDataSets][rotImg][listVar] = mask[listVar]
            
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
            opening[listVar] = cv2.morphologyEx(mask[listVar], cv2.MORPH_OPEN, kernel, iterations=0) 
            close[listVar] = cv2.morphologyEx(opening[listVar], cv2.MORPH_CLOSE, kernel, iterations=25)
            cnts = cv2.findContours(close[listVar], cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            for c in cnts:
                area = cv2.contourArea(c)
                if area > conSizeList[listVar] and meanSizeList[listVar] > 0:
                    cells_count[listVar] += math.ceil(area / meanSizeList[listVar])
                elif meanSizeList[listVar] > 0:
                    cells_count[listVar] += 1
                else:
                    cells_count[listVar] += 0
                   
        count_live[rotDataSets].append(cells_count[0])
        count_early[rotDataSets].append(cells_count[1])
        count_late[rotDataSets].append(cells_count[2])
        count_nec[rotDataSets].append(cells_count[3])

for rotDataSets in range(numDataSets):
    live_means.append(s.mean(count_live[rotDataSets]))
    live_stdevs.append(s.stdev(count_live[rotDataSets]))
    early_means.append(s.mean(count_early[rotDataSets]))
    early_stdevs.append(s.stdev(count_early[rotDataSets]))
    late_means.append(s.mean(count_late[rotDataSets]))
    late_stdevs.append(s.stdev(count_late[rotDataSets]))
    nec_means.append(s.mean(count_nec[rotDataSets]))
    nec_stdevs.append(s.stdev(count_nec[rotDataSets]))
    
    totals.append(live_means[rotDataSets]+early_means[rotDataSets]+late_means[rotDataSets]+nec_means[rotDataSets])
    if(totals[rotDataSets] > 0):
        early_per.append(early_means[rotDataSets]/totals[rotDataSets]*100)
        late_per.append(late_means[rotDataSets]/totals[rotDataSets]*100)
        nec_per.append(nec_means[rotDataSets]/totals[rotDataSets]*100)
    print("Total Cells counted in well: ", math.ceil(totals[rotDataSets]*5), "\n")
    
print("-----------STATS-----------")
if(len(early_per) > 2):
    print("\nEarly Apoptotic % Mean/Stdev: ", s.mean(early_per), "+/-", s.stdev(early_per)/math.sqrt(numImg))
    print("Late Apoptotic % Mean/Stdev: ", s.mean(late_per), "+/-", s.stdev(late_per))
    print("Necrotic % Mean/Stdev: ", s.mean(nec_per), "+/-", s.stdev(nec_per)/math.sqrt(numImg))
    print("Total % Mean/Stdev: ", s.mean(early_per)+s.mean(late_per)+s.mean(nec_per), "+/-", math.sqrt(s.stdev(early_per)**2 + s.stdev(late_per)**2 + s.stdev(nec_per))/math.sqrt(numDataSets))